# Batch-wise Structure Relaxation

In this tutorial, we show how to use the ``TorchStructureLBFGS``. It enables relaxation of structures in a batch-wise manner, i.e. it optimizes multiple structures in parallel. This is particularly useful, when many relatively similar structures (--> similar time until convergence) should be relaxed while requiring possibly short simulation time.

In [2]:
import torch
from ase.io import read

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter
from schnetpack.interfaces.batchwise_optimizer import TorchStructureLBFGS

First, we load the force field model that provides the forces for the relaxation process. To avoids back-propagation through the graph multiple times, the response (forces) module is disabled.

In [5]:
model_path = "../../tests/testdata/md_ethanol.model"

# set device
device = torch.device("cuda")

# load model
model = torch.load(model_path, map_location=device)

# remove response modules (to avoid differentiating more than once)
model.model_outputs = ["energy"]
model.do_postprocessing = False
model.required_derivatives = []
model.output_modules = torch.nn.ModuleList([model.output_modules[0]])

Subsequently, we load the batch of initial structures utilizing ASE (supports xyz, db and more) and convert it to SchNetPack input format. For this purpose we need use the ``AtomsConverter`` with suitable neighbor list.

In [6]:
input_structure_file = "../../tests/testdata/md_ethanol.xyz"

# load initial structures
ats = read(input_structure_file, index=":")

# define neighbor list
cutoff = model.representation.cutoff.item()
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# build atoms converter
atoms_converter = AtomsConverter(
    neighbor_list=nbh_list,
    device=device,
)

# convert atoms object to schnetpack batch
inputs = atoms_converter(ats)

For some systems it helps to fix the positions of certain atoms during the relaxation. This can be achieved by providing a mask of boolean entries to ``TorchStructureLBFGS``. The mask is a list of $3n_\text{atoms}$ entries, i.e. it contains three entries for each atom associated with the respective directions in cartesian space. Here, we do not fix any atoms. Hence, the mask only contains ``True``.

In [7]:
# define structure mask for optimization (True for fixed, False for non-fixed)
n_atoms = len(ats[0].get_atomic_numbers())
single_structure_mask = [False for _ in range(n_atoms * 3)]
# expand mask by number of input structures (fixed atoms are equivalent for all input structures)
mask = single_structure_mask * len(ats)

Finally, we run the optimization:

In [8]:
# Initialize optimizer
optimizer = TorchStructureLBFGS(
    model=model,
    model_inputs=inputs,
    fixed_atoms_mask=mask
)

# run optimization
optimizer.run(fmax=0.0005, max_opt_steps=1000)

  0%|          | 2/1000 [00:00<04:11,  3.96it/s]
INFO:root:max. atomic force: 0.0003327234007883817


In [9]:
# get new atomic positions
optimizer.get_relaxed_structure()

tensor([[-4.9264,  1.5385, -0.0645],
        [-3.4131,  1.4534, -0.1381],
        [-5.2331,  2.2933,  0.6734],
        [-5.3542,  0.5698,  0.2303],
        [-5.3460,  1.8166, -1.0417],
        [-2.9984,  1.1848,  0.8521],
        [-2.9902,  2.4354, -0.4237],
        [-3.0735,  0.4551, -1.1144],
        [-2.1046,  0.3996, -1.1626]], device='cuda:0', requires_grad=True)